In [1]:
import time 

from IPython.core.display import display
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.style.use('ggplot')
import numpy as np
import scipy
import seaborn as sns
sns.set(font='Osaka')
import pandas as pd

In [2]:
t0 = time.time()
data_dir = '../data/'
file = 'df_item_transaction.csv'
df_item_transaction = pd.read_csv(data_dir + file)
print('Load in ', time.time() - t0)

Load in  1.4672930240631104


In [3]:
country = 'United Kingdom'
df_uk = \
df_item_transaction[df_item_transaction['Country'] == country].drop('Unnamed: 0', axis=1)

# CustomerID to Nominal
df_uk['CustomerID'] = df_uk['CustomerID'].fillna('NaN')

# InvoiceDate to Datetime
df_uk['InvoiceDate'] = pd.to_datetime(df_uk['InvoiceDate'])

assert df_uk['StockCode'].nunique() == 3910
assert df_uk['CustomerID'].nunique() == 3918

print('# of distinct items =', df_uk['StockCode'].nunique())
print('# of distinct customers =', df_uk['CustomerID'].nunique())

# of distinct items = 3910
# of distinct customers = 3918


In [19]:
# Customer ごとにデータを集計　
# Recency
# Frequency = #ofTransactions
# Monetary = sum(Sales)

# Sales: min, max, median, mean, sum
# Quantity: min, max, median, mean, sum
# Quantity / Transaction: min, max, median, mean
# Unitprice: min, max, median, mean

# Item / Transaction = 商品の種類 / Transaction: min, max, median, mean


# 各商品をいくつ購入したか？　(3,910 items hence the same # of columns)

# TODO:
# ある商品を購入する顧客は Frequency が大きいといった関係が発見できるか？
# RMF 分析で優良顧客を見つけ出し、彼らの 2月, 4月の


# Monetary
df_uk_per_customer = \
pd.DataFrame(df_uk.groupby('CustomerID')['Sales'].sum()).reset_index()\
.rename(columns={'Sales': 'Monetary'})

# Frequency
df_uk_per_customer['Frequency'] = \
pd.DataFrame(df_uk.groupby('CustomerID')['InvoiceNo'].nunique()).reset_index()['InvoiceNo']
# small test
test_customer = 12748
assert df_uk_per_customer[df_uk_per_customer['CustomerID'] == test_customer]['Frequency'].values\
            == df_uk[df_uk['CustomerID'] == test_customer]['InvoiceNo'].nunique()

# Recency
# 基準日
df_uk_per_customer['ReferenceDate'] = \
    pd.to_datetime(df_item_transaction['InvoiceDate']).max()
# 最終購入日
df_uk_per_customer['LastTransactionDate'] = \
    pd.DataFrame(df_uk.groupby('CustomerID')['InvoiceDate'].max()).reset_index()['InvoiceDate']
# 基準日 - 最終購入日
df_uk_per_customer['Recency(Day)'] = \
(df_uk_per_customer['ReferenceDate'].dt.date - df_uk_per_customer['LastTransactionDate'].dt.date).dt.days


# sum(Quantity)
df_uk_per_customer['QuantitySum'] = pd.DataFrame(df_uk.groupby('CustomerID')['Quantity'].sum()).reset_index()['Quantity']

# Quantity / Transaction
df_uk_per_customer['QuantityPerTransaction'] = df_uk_per_customer['QuantitySum'] / df_uk_per_customer['Frequency']

# Sales / Transaction
df_uk_per_customer['Sales/Transaction'] = df_uk_per_customer['Monetary'] / df_uk_per_customer['Frequency']

# Drop CustomerID = NaN
df_uk_per_customer = df_uk_per_custome[df_uk_per_customer['CustomerID'] != 'NaN']

NameError: name 'df_uk_per_custome' is not defined

In [5]:
# User-Item matrix

# CustomerID = NaN を除外
df_uk = df_uk[df_uk['CustomerID'] != 'NaN']

df_uk_user_item_matrix = \
pd.DataFrame(df_uk.groupby(['CustomerID', 'StockCode'])['Quantity'].sum())\
.reset_index()\
.pivot(index='CustomerID', columns='StockCode', values='Quantity')\
.fillna(0)

In [17]:
df_uk_user_item_matrix.head(3)

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z,BANK CHARGES,PADS
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12747.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12748.0,1.0,0.0,6.0,0.0,0.0,0.0,0.0,28.0,36.0,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Logistic Regression 

In [20]:
df_uk_per_customer.loc[df_uk_per_customer['Frequency'] <= 2, 'OverF2'] = 0
df_uk_per_customer.loc[df_uk_per_customer['Frequency'] > 2, 'OverF2'] = 1
df_uk_per_customer['OverF2'] = df_uk_per_customer['OverF2'].astype(np.int)

In [21]:
display(df_uk_per_customer.tail(3))

,CustomerID,Monetary,Frequency,ReferenceDate,LastTransactionDate,Recency(Day),QuantitySum,QuantityPerTransaction,Sales/Transaction,OverF2
3914,18282.0,178.05,2,2011-12-09 12:50:00,2011-12-02 11:43:00,7,103,51.500000,89.025000,0
3915,18283.0,2088.93,16,2011-12-09 12:50:00,2011-12-06 12:02:00,3,1395,87.187500,130.558125,1
3916,18287.0,1837.28,3,2011-12-09 12:50:00,2011-10-28 09:29:00,42,1586,528.666667,612.426667,1


In [11]:
df_uk_per_customer.groupby('OverF2').size()

OverF2
0.0    2096
1.0    1822
dtype: int64

In [22]:
from sklearn.linear_model import LogisticRegression

X = df_uk_user_item_matrix.values
y = df_uk_per_customer['OverF2'].values

lr = LogisticRegression(penalty='l1')

lr.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [46]:
df_coef.T.sort_values(0, ascending=False).head(10)

,0
StockCode,
23702,1.449290
79062D,1.242978
21476,0.905490
85135C,0.858054
22513,0.848836
22215,0.823958
23021,0.815616
21333,0.701041
84709B,0.679516


In [36]:
file = 'df_item_description.csv'
df_item_description = pd.read_csv(data_dir + file)

In [43]:
for stock_code in df_coef.T.sort_values(0, ascending=False).head(10).index:
    print(stock_code, '\t' + df_item_description[df_item_description['StockCode'] == stock_code]['Description'].values[0])

23702 	High Resolution Image
79062D 	RETRO TIN ASHTRAY,REVOLUTIONARY
21476 	STEEL SWEETHEART ROUND TABLE CREAM
85135C 	RED DRAGONFLY HELICOPTER
22513 	DOORSTOP FOOTBALL DESIGN
22215 	CAKE STAND WHITE TWO TIER LACE
23021 	GLASS BONNE JAM JAR
21333 	CLASSIC WHITE FRAME
84709B 	PINK OVAL JEWELLED MIRROR
23406 	HOME SWEET HOME KEY HOLDER


## Logistic Regression を丁寧に

In [47]:
X = df_uk_user_item_matrix.values
y = df_uk_per_customer['OverF2'].values
print(X.shape)
print(y.shape)

(3917, 3641)
(3917,)


In [49]:
# Test data: 25%
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0, stratify=y)
print(X_train.shape)
print(X_test.shape)

(2937, 3641)
(980, 3641)


In [54]:
# 10-Fold Cross Validation
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

coef_dict = {}
df_cv = pd.DataFrame()

n_splits = 10
kf = KFold(n_splits, shuffle=True, random_state=0)

for i, (train_index, test_index) in enumerate(kf.split(X_train)):
    X_tr, X_val = X_train[train_index], X_train[test_index]
    y_tr, y_val = y_train[train_index], y_train[test_index]
    
    lr = LogisticRegression(penalty='l1')
    lr.fit(X_tr, y_tr)
    y_pred = lr.predict(X_val)
    df_cv.loc[i, 'cv_score'] = accuracy_score(y_val, y_pred)
    
    coef_dict['Fold_{}'.format(i + 1)] = lr.coef_[0]

In [61]:
df_cv.T

,0,1,2,3,4,5,6,7,8,9
cv_score,0.778912,0.734694,0.734694,0.809524,0.738095,0.795918,0.744898,0.798635,0.757679,0.78157


In [67]:
df_cv['cv_score'].mean()

0.76746186529219207

In [64]:
coef_dict['Fold_1']

array([[-0.03555901,  0.        , -0.03409742, ...,  0.        ,
         0.        ,  0.        ]])

In [66]:
lr.coef_[0]

array([-0.09224137, -0.0210183 , -0.01500303, ...,  0.        ,
        0.        ,  0.        ])

## Logistic Regression
-  L2

In [68]:
# 10-Fold Cross Validation
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

coef_dict = {}
df_cv = pd.DataFrame()

n_splits = 10
kf = KFold(n_splits, shuffle=True, random_state=0)

for i, (train_index, test_index) in enumerate(kf.split(X_train)):
    X_tr, X_val = X_train[train_index], X_train[test_index]
    y_tr, y_val = y_train[train_index], y_train[test_index]
    
    lr = LogisticRegression(penalty='l2')
    lr.fit(X_tr, y_tr)
    y_pred = lr.predict(X_val)
    df_cv.loc[i, 'cv_score'] = accuracy_score(y_val, y_pred)
    
    coef_dict['Fold_{}'.format(i + 1)] = lr.coef_[0]

In [70]:
df_cv.T

,0,1,2,3,4,5,6,7,8,9
cv_score,0.64966,0.683673,0.653061,0.704082,0.676871,0.714286,0.680272,0.658703,0.658703,0.665529


## Logistic Regression
- L1
- C = 0.1

In [73]:
# 10-Fold Cross Validation
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

penalty = 'l1'
C = 0.1

coef_dict = {}
df_cv = pd.DataFrame()

n_splits = 10
kf = KFold(n_splits, shuffle=True, random_state=0)

for i, (train_index, test_index) in enumerate(kf.split(X_train)):
    X_tr, X_val = X_train[train_index], X_train[test_index]
    y_tr, y_val = y_train[train_index], y_train[test_index]
    
    lr = LogisticRegression(penalty=penalty, C=C)
    lr.fit(X_tr, y_tr)
    y_pred = lr.predict(X_val)
    df_cv.loc[i, 'cv_score'] = accuracy_score(y_val, y_pred)
    
    coef_dict['Fold_{}'.format(i + 1)] = lr.coef_[0]

In [75]:
df_cv['cv_score'].mean()

0.8021859255647652

## Logistic Regression
- L1
- C = 0.01

In [76]:
# 10-Fold Cross Validation
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

penalty = 'l1'
C = 0.01

coef_dict = {}
df_cv = pd.DataFrame()

n_splits = 10
kf = KFold(n_splits, shuffle=True, random_state=0)

for i, (train_index, test_index) in enumerate(kf.split(X_train)):
    X_tr, X_val = X_train[train_index], X_train[test_index]
    y_tr, y_val = y_train[train_index], y_train[test_index]
    
    lr = LogisticRegression(penalty=penalty, C=C)
    lr.fit(X_tr, y_tr)
    y_pred = lr.predict(X_val)
    df_cv.loc[i, 'cv_score'] = accuracy_score(y_val, y_pred)
    
    coef_dict['Fold_{}'.format(i + 1)] = lr.coef_[0]

In [77]:
df_cv['cv_score'].mean()

0.79912354020106335

## Logistic Regression
- L1
- C = 0.05

In [78]:
# 10-Fold Cross Validation
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

penalty = 'l1'
C = 0.05

coef_dict = {}
df_cv = pd.DataFrame()

n_splits = 10
kf = KFold(n_splits, shuffle=True, random_state=0)

for i, (train_index, test_index) in enumerate(kf.split(X_train)):
    X_tr, X_val = X_train[train_index], X_train[test_index]
    y_tr, y_val = y_train[train_index], y_train[test_index]
    
    lr = LogisticRegression(penalty=penalty, C=C)
    lr.fit(X_tr, y_tr)
    y_pred = lr.predict(X_val)
    df_cv.loc[i, 'cv_score'] = accuracy_score(y_val, y_pred)
    
    coef_dict['Fold_{}'.format(i + 1)] = lr.coef_[0]

In [79]:
df_cv['cv_score'].mean()

0.81138004689930587

In [80]:
# Test
y_pred = lr.predict(X_test)
print('Test accuracy = ', accuracy_score(y_test, y_pred))

Test accuracy =  0.810204081633


In [83]:
df_coef = \
pd.DataFrame.from_dict(coef_dict).T
df_coef.columns = df_uk_user_item_matrix.columns

In [93]:
i = 121
df_coef.iloc[:, i:i + 20]

StockCode,20654,20655,20657,20658,20659,20661,20662,20663,20664,20665,20666,20667,20668,20669,20670,20671,20674,20675,20676,20677
Fold_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.003666,0.0,0.0,0.0,0.0,0.052462,0.0,0.029529
Fold_10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.003514,0.0,0.0,0.0,0.0,0.049875,0.0,0.025115
Fold_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.003154,0.0,0.0,0.0,0.0,0.048684,0.0,0.045340
Fold_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.023660,0.0,0.000000
Fold_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000723,0.0,0.0,0.0,0.0,0.084392,0.0,0.000000
Fold_5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.003008,0.0,0.0,0.0,0.0,0.038117,0.0,0.045262
Fold_6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.002480,0.0,0.0,0.0,0.0,0.000000,0.0,0.074138
Fold_7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000921,0.0,0.0,0.0,0.0,0.046114,0.0,0.000000
Fold_8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.001862,0.0,0.0,0.0,0.0,0.030857,0.0,0.004493
Fold_9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.001422,0.0,0.0,0.0,0.0,0.038646,0.0,0.024640


In [95]:
df_coef.loc['avg'] = df_coef.mean(axis=0)

In [97]:
df_coef.T.sort_values('avg', ascending=False)

,Fold_1,Fold_10,Fold_2,Fold_3,Fold_4,Fold_5,Fold_6,Fold_7,Fold_8,Fold_9,avg
StockCode,,,,,,,,,,,
21034,0.134226,0.155960,0.171304,0.161814,0.148723,0.147726,0.146941,0.148455,0.208131,0.103317,0.152660
22568,0.121303,0.171559,0.143581,0.135924,0.177697,0.128595,0.158345,0.149634,0.155183,0.135207,0.147703
72807A,0.154278,0.000000,0.194142,0.159363,0.170441,0.125961,0.153029,0.156510,0.181049,0.143038,0.143781
22900,0.139719,0.138169,0.162789,0.135225,0.136329,0.129311,0.127782,0.107371,0.138626,0.086171,0.130149
22726,0.020543,0.091588,0.123958,0.165138,0.108320,0.131928,0.176961,0.104552,0.118185,0.084058,0.112523
82486,0.134535,0.089607,0.144752,0.120576,0.121569,0.079683,0.085506,0.124765,0.149507,0.064018,0.111452
22607,0.087003,0.118773,0.014902,0.124242,0.140176,0.119147,0.125550,0.129750,0.119455,0.098985,0.107798
22570,0.150930,0.058191,0.117002,0.096025,0.067166,0.091043,0.113691,0.054957,0.119809,0.066452,0.093527
21086,0.071984,0.038894,0.123255,0.103487,0.091241,0.112288,0.120177,0.089853,0.077536,0.062762,0.089148


In [98]:
for stock_code in df_coef.T.sort_values('avg', ascending=False).head(10).index:
    print(stock_code, '\t' + df_item_description[df_item_description['StockCode'] == stock_code]['Description'].values[0])

21034 	REX CASH+CARRY JUMBO SHOPPER
22568 	FELTCRAFT CUSHION OWL
72807A 	SET/3 ROSE CANDLE IN JEWELLED BOX
22900 	SET 2 TEA TOWELS I LOVE LONDON 
22726 	ALARM CLOCK BAKELIKE GREEN
82486 	WOOD S/3 CABINET ANT WHITE FINISH
22607 	WOODEN ROUNDERS GARDEN SET 
22570 	FELTCRAFT CUSHION RABBIT
21086 	SET/6 RED SPOTTY PAPER CUPS
22429 	ENAMEL MEASURING JUG CREAM
